<a href="https://colab.research.google.com/github/SepideSohrabi/K-fold-Image-Classification/blob/main/PyTprch_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
#C:\Users\sadma\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\client\session.py:1766: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  #warnings.warn('An interactive session is already active. This can '

In [2]:


# Make it False if do not intend to use Google Colab and want to train in local machine!!
google_colab_flag = True

# For training in Google Colab
if(google_colab_flag):
    from google.colab import drive
    drive.mount('/content/drive')
    !ls
    import sys
    # This is the path to where in google drive the code is stored!
    
    root_path = '/content/drive/MyDrive/AmnAzmoonPics/'
    sys.path.append(root_path)

# For local training
else:
    root_path = ''



Mounted at /content/drive
drive  sample_data


In [3]:
from PIL import Image
import glob
image_legal = []
for filename in glob.glob('/content/drive/MyDrive/AmnAzmoonPics/Legal/*.png'): #assuming png
    im=Image.open(filename)
    image_legal.append(im)

In [4]:
image_ilegal = []
for filename in glob.glob('/content/drive/MyDrive/AmnAzmoonPics/iLegal/*.png'): #assuming png
    im=Image.open(filename)
    image_ilegal.append(im)

In [5]:
#Rotating the images in image_legal and appending them to the list
for i in range (len(image_legal)):
  rotated_img = image_legal[i].rotate(90)
  image_legal.append(rotated_img)

In [6]:
#Rotating the images in image_ilegal and appending them to the list
for i in range (len(image_ilegal)):
  rotated_img = image_ilegal[i].rotate(90)
  image_ilegal.append(rotated_img)

In [7]:
image_ilegal[90].size

(2736, 1824)

In [8]:
#Cropping the images in image_legal and appending them to the list
box = (100, 100, 1800, 1500)
for i in range (len(image_legal)):
  Cropped_img = image_legal[i].crop(box)
  image_legal.append(Cropped_img)


In [9]:
 #Cropping the images in image_ilegal and appending them to the list
 for i in range (len(image_ilegal)):
  Cropped_img = image_ilegal[i].crop(box)
  image_ilegal.append(Cropped_img)

In [ ]:
len(image_legal)

232

In [ ]:
image_ilegal[120]

In [10]:
#for preprocessing 
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])


#خط [۱]: در اینجا ما یک تغییر متغیر تعریف می کنیم که ترکیبی از تمام تبدیلات تصویر است که باید روی تصویر ورودی اعمال شود.

#خط [۲]: اندازه تصویر را به ۲۵۶ × ۲۵۶ پیکسل تغییر می دهد.

#خط [۳]: تصویر را به ۲۲۴ × ۲۲۴ پیکسل از مرکز برش می دهد.

#خط [۴]: تصویر را به نوع داده Pythorch Tensor تبدیل می کند.

#خط [۵-۷]: با تنظیم میانگین و انحراف معیار آن روی مقادیر مشخص شده ، تصویر را نرمال می کند

In [11]:
X_iLegal=[]
for img in image_ilegal:
    transformed_img=transform(img)
    X_iLegal.append(transformed_img)


In [12]:
X_Legal=[]
for img in image_legal:
  transformed_img=transform(img)
  X_Legal.append(transformed_img)

In [ ]:
"""import torch 
# Load the ResNext-WSL model

# small size model
#model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
# medium size
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')
# large size
# model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x32d_wsl')
# very large
# model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x48d_wsl')
# model.eval()

Downloading: "https://github.com/facebookresearch/WSL-Images/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x16-c6f796b0.pth


  0%|          | 0.00/741M [00:00<?, ?B/s]

In [13]:
#Define the CCN
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

# Define a convolution neural network
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*10*10, 10)

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))     
        output = output.view(-1, 24*10*10)
        output = self.fc1(output)

        return output

# Instantiate a neural network model 
model = Network()

In [15]:
from torch.optim import Adam
 
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [16]:
#Train the model on the training data
from torch.autograd import Variable

# Function to save the model
def saveModel():
    path = "./myFirstModel.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 999:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [14]:
#تصویر ورودی را بارگیری کرده و تبدیلات تصویری را که در بالا مشخص کردیم انجام دهیم#
#تصویر را پیش پردازش کرده و دسته ای را برای انتقال از شبکه آماده می کنیم.
model.eval()
out_Legal=[]
for i in range (len(X_Legal)):
    batch_t = torch.unsqueeze(X_Legal[i], 0)
    out_Legal.append(model(batch_t).detach().numpy())



RuntimeError: ignored

In [ ]:
out_iLegal=[]
for i in range (len(X_iLegal)):
    batch_t = torch.unsqueeze(X_iLegal[i], 0)
    out_iLegal.append(model(batch_t).detach().numpy())

In [ ]:
out_Legal_array=np.asarray(out_Legal)
out_iLegal_array=np.asarray(out_iLegal)

In [ ]:
out_iLegal_array.shape

(232, 1, 1000)

In [ ]:
X_train=np.concatenate((out_Legal_array,out_iLegal_array) , axis=0)
X_train=np.asarray(X_train)

In [ ]:
y_train=[]
for i in range (232):
  legal=1
  y_train.append(legal)


for i in range (232 ):
  ilegal=0
  y_train.append(ilegal)

In [ ]:
y_train=np.asarray(y_train)

In [ ]:
X_train=np.squeeze(X_train)

In [ ]:
y_train


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=3)
   
#hist = model.fit(x_train, y_train, epochs=1000, batch_size=32, 
                 #validation_data=(x_val, y_val), callbacks=[early_stopping])


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train , y_train )

SVC()

In [ ]:
# K-Fold Cross-Validation
from sklearn.model_selection import cross_validate
def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = ['accuracy', 'precision', 'recall', 'f1']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training Accuracy scores": results['train_accuracy'],
              
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training Precision scores": results['train_precision'],
              "Mean Training Precision": results['train_precision'].mean(),
              "Training Recall scores": results['train_recall'],
              "Mean Training Recall": results['train_recall'].mean(),
              "Training F1 scores": results['train_f1'],
              "Mean Training F1 Score": results['train_f1'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation Precision scores": results['test_precision'],
              "Mean Validation Precision": results['test_precision'].mean(),
              "Validation Recall scores": results['test_recall'],
              "Mean Validation Recall": results['test_recall'].mean(),
              "Validation F1 scores": results['test_f1'],
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

In [ ]:
cross_validation(clf, X_train, y_train, _cv=5)

{'Training Accuracy scores': array([0.77088949, 0.77897574, 0.7574124 , 0.79245283, 0.77419355]),
 'Mean Training Accuracy': 77.47848013216243,
 'Training Precision scores': array([0.77173913, 0.74178404, 0.75806452, 0.77114428, 0.73831776]),
 'Mean Training Precision': 0.7562099439477622,
 'Training Recall scores': array([0.76756757, 0.85405405, 0.75806452, 0.83333333, 0.84946237]),
 'Mean Training Recall': 0.812496367335077,
 'Training F1 scores': array([0.7696477 , 0.79396985, 0.75806452, 0.80103359, 0.79      ]),
 'Mean Training F1 Score': 0.7825431307166989,
 'Validation Accuracy scores': array([0.67741935, 0.74193548, 0.7311828 , 0.55913978, 0.64130435]),
 'Mean Validation Accuracy': 67.01963534361852,
 'Validation Precision scores': array([0.9047619 , 0.84848485, 0.78378378, 0.53846154, 0.5915493 ]),
 'Mean Validation Precision': 0.7334082742533446,
 'Validation Recall scores': array([0.40425532, 0.59574468, 0.63043478, 0.76086957, 0.91304348]),
 'Mean Validation Recall': 0.6608

In [ ]:
clf = GridSearchCV(model, parameters,
                         fit_params={'early_stopping_rounds':20,\
                         'eval_set':[(X,y)]},cv=kfold)

In [ ]:
# Visualising the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green', 'blue')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green', 'blue'))(i), label = j)
plt.title('Kernel SVM (Training set)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.show()

In [ ]:
# Grouped Bar Chart for both training and validation data
def plot_result(x_label, y_label, plot_title, train_data, val_data):
        '''Function to plot a grouped bar chart showing the training and validation
          results of the ML model in each fold after applying K-fold cross-validation.
         Parameters
         ----------
         x_label: str, 
            Name of the algorithm used for training e.g 'Decision Tree'
          
         y_label: str, 
            Name of metric being visualized e.g 'Accuracy'
         plot_title: str, 
            This is the title of the plot e.g 'Accuracy Plot'
         
         train_result: list, array
            This is the list containing either training precision, accuracy, or f1 score.
        
         val_result: list, array
            This is the list containing either validation precision, accuracy, or f1 score.
         Returns
         -------
         The function returns a Grouped Barchart showing the training and validation result
         in each fold.
        '''
        
        # Set size of plot
        plt.figure(figsize=(12,6))
        labels = ["1st Fold", "2nd Fold", "3rd Fold", "4th Fold", "5th Fold"]
        X_axis = np.arange(len(labels))
        ax = plt.gca()
        plt.ylim(0.40000, 1)
        plt.bar(X_axis-0.2, train_data, 0.4, color='blue', label='Training')
        plt.bar(X_axis+0.2, val_data, 0.4, color='red', label='Validation')
        plt.title(plot_title, fontsize=30)
        plt.xticks(X_axis, labels)
        plt.xlabel(x_label, fontsize=14)
        plt.ylabel(y_label, fontsize=14)
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [ ]:
#First, check if test folder is empty or not, if not transfer all existing files to train.


In [ ]:
transfer_all_class_between_folders('test', 'train', 1.0)

No file moved. Source empty!
No file moved. Source empty!


In [ ]:
#Now, split some part of train data into the test folders.

In [ ]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [ ]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [ ]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [ ]:


X = np.asarray(X)
Y = np.asarray(Y)



In [ ]:
batch_size = 512
epoch = 100 #اینجا رو از صد گذاشتم ده که زمان نگیره 
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [ ]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()

In [ ]:
#Stratified K-Fold Cross validation

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train', 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 62 images belonging to 2 classes.
Found 32 images belonging to 3 classes.
Epoch 1/100
1/1 [==============================] - 10s 10s/step - loss: 0.6921 - accuracy: 0.4839
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 0.6985 - accuracy: 0.5484
Epoch 3/100
1/1 [==============================] - 9s 9s/step - loss: 0.6937 - accuracy: 0.5484
Epoch 4/100
1/1 [==============================] - 8s 8s/step - loss: 0.6864 - accuracy: 0.5323
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 0.7017 - accuracy: 0.5161
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 0.7056 - accuracy: 0.4839
Epoch 7/100
1/1 [==============================] - 11s 11s/step - loss: 0.6956 - accuracy: 0.5323
Epoch 8/100
1/1 [==============================] - 8s 8s/step - loss: 0.6864 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 0.7057 - accuracy: 0.4516
Epoch 10/100
1/1 [===============

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 31 images belonging to 3 classes.
Epoch 1/100
1/1 [==============================] - 8s 8s/step - loss: 0.5616 - accuracy: 0.7619
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 0.4981 - accuracy: 0.8095
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 0.6409 - accuracy: 0.7619
Epoch 4/100
1/1 [==============================] - 8s 8s/step - loss: 0.5410 - accuracy: 0.7937
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 0.5188 - accuracy: 0.7619
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 0.5974 - accuracy: 0.7143
Epoch 7/100
1/1 [==============================] - 8s 8s/step - loss: 0.4689 - accuracy: 0.7937
Epoch 8/100
1/1 [==============================] - 10s 10s/step - loss: 0.4710 - accuracy: 0.8095
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 0.4918 - accuracy: 0.7619
Epoch 10/100
1/1 [==============================] - 8s 8s/step - loss: 0.4937 - accuracy: 0.73

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/100
1/1 [==============================] - 9s 9s/step - loss: 0.4515 - accuracy: 0.8254
Epoch 2/100
1/1 [==============================] - 8s 8s/step - loss: 0.3354 - accuracy: 0.8413
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 0.3613 - accuracy: 0.8095
Epoch 4/100
1/1 [==============================] - 8s 8s/step - loss: 0.3259 - accuracy: 0.8413
Epoch 5/100
1/1 [==============================] - 8s 8s/step - loss: 0.3343 - accuracy: 0.8254
Epoch 6/100
1/1 [==============================] - 8s 8s/step - loss: 0.3979 - accuracy: 0.7460
Epoch 7/100
1/1 [==============================] - 8s 8s/step - loss: 0.2864 - accuracy: 0.8254
Epoch 8/100
1/1 [==============================] - 8s 8s/step - loss: 0.2921 - accuracy: 0.8730
Epoch 9/100
1/1 [==============================] - 8s 8s/step - loss: 0.3631 - accuracy: 0.8571
Epoch 10/100
1/1 [==============================] - 8s 8s/step - loss: 0.3650 - accuracy: 0.8254
Epoch 11/100
1/1 [=====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

# save the model to disk
import pickle
filename = 'finalized_model.h5'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

SyntaxError: ignored

In [ ]:
import cv2

In [ ]:
img = (cv2.imread("/content/8.png"))/255.0

In [ ]:
img = np.resize(img ,(1,100,100,3))

In [ ]:
model.predict(img)

1/1 [==============================] - 0s 40ms/step


array([[0.19827849, 0.8017215 ]], dtype=float32)